# Load Data từ Bronze Layer sang Silver Layer

Notebook này sẽ đọc dữ liệu từ Bronze layer (MinIO) và xử lý để load vào các bảng Iceberg trong Silver layer với Nessie catalog.

## 1. Import Libraries và Khởi tạo Spark Session

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window
import csv, io, os, re
from datetime import datetime
from typing import Dict

# Cấu hình AWS/MinIO credentials
os.environ.update({
    'AWS_REGION': 'us-east-1',
    'AWS_ACCESS_KEY_ID': 'admin',
    'AWS_SECRET_ACCESS_KEY': 'admin123'
})

# Khởi tạo Spark Session với Nessie Catalog
spark = (
    SparkSession.builder
    .appName("Load_Bronze_To_Silver")
    .master("spark://spark-master:7077")
    .config("spark.executor.memory", "2g")
    .config("spark.executor.cores", "2")
    # Nessie Catalog
    .config("spark.sql.catalog.nessie", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.nessie.catalog-impl", "org.apache.iceberg.nessie.NessieCatalog")
    .config("spark.sql.catalog.nessie.uri", "http://nessie:19120/api/v2")
    .config("spark.sql.catalog.nessie.ref", "main")
    .config("spark.sql.catalog.nessie.warehouse", "s3a://silver/")
    .config("spark.sql.catalog.nessie.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
    # S3/MinIO Config
    .config("spark.sql.catalog.nessie.s3.endpoint", "http://minio:9000")
    .config("spark.sql.catalog.nessie.s3.access-key-id", "admin")
    .config("spark.sql.catalog.nessie.s3.secret-access-key", "admin123")
    .config("spark.sql.catalog.nessie.s3.path-style-access", "true")
    .config("spark.sql.catalog.nessie.s3.region", "us-east-1")
    # Hadoop S3A Config
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
    .config("spark.hadoop.fs.s3a.access.key", "admin")
    .config("spark.hadoop.fs.s3a.secret.key", "admin123")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
    .config("spark.hadoop.fs.s3a.region", "us-east-1")
    # Executor Environment
    .config("spark.executorEnv.AWS_REGION", "us-east-1")
    .config("spark.executorEnv.AWS_ACCESS_KEY_ID", "admin")
    .config("spark.executorEnv.AWS_SECRET_ACCESS_KEY", "admin123")
    # Local JAR files
    .config("spark.jars", "/opt/spark/jars/hadoop-aws-3.3.4.jar,/opt/spark/jars/aws-java-sdk-bundle-1.12.262.jar")
    .getOrCreate()
)

spark.sparkContext.setLogLevel("ERROR")
spark.sql("CREATE DATABASE IF NOT EXISTS nessie.silver_tables")
spark.sql("USE nessie.silver_tables")
print(f" Spark Session initialized | Master: {spark.sparkContext.master} | App ID: {spark.sparkContext.applicationId}")


 Spark Session initialized | Master: spark://spark-master:7077 | App ID: app-20251205101919-0001


## 2. Load Bảng SCHOOL

In [ ]:
# LOAD SCHOOL: Danh sách trường đại học từ 2021-2025
base_path = "s3a://bronze/structured_data/danh sách các trường Đại Học (2021-2025)/Danh_sách_các_trường_Đại_Học_"
df_school = (
    spark.read.option("header", "true").option("inferSchema", "true")
    .csv([f"{base_path}{year}.csv" for year in range(2021, 2026)])
    .select("TenTruong", "MaTruong", "TinhThanh")
    .dropDuplicates()
)

df_school_silver = df_school.select(
    col("MaTruong").cast("string").alias("schoolId"),
    col("TenTruong").cast("string").alias("schoolName"),
    col("TinhThanh").cast("string").alias("province"),
    current_timestamp().alias("created_at"),
    current_timestamp().alias("updated_at")
).filter(col("schoolId").isNotNull() & col("schoolName").isNotNull())

df_school_silver.writeTo("nessie.silver_tables.school").using("iceberg").createOrReplace()
print(f"✅ SCHOOL: {df_school_silver.count()} records")

## 3. Load Bảng MAJOR

In [ ]:
# LOAD MAJOR: Danh sách ngành đại học, dedupe theo lowercase majorId
df_major = (
    spark.read.option("header", "true").option("encoding", "UTF-8")
    .csv("s3a://bronze/structured_data/danh sách các ngành đại học/Danh_sách_các_ngành.csv")
)

df_major_silver = (
    df_major.select(
        regexp_replace(trim(col(df_major.columns[0])), r"\.0$", "").alias("majorId"),
        trim(col(df_major.columns[1])).alias("majorName")
    )
    .filter(
        col("majorId").isNotNull() & (col("majorId") != "") &
        col("majorName").isNotNull() & (col("majorName") != "") &
        (lower(col("majorId")) != "nan")
    )
    .withColumn("majorId_lower", lower(col("majorId")))
    .dropDuplicates(["majorId_lower"])
    .select(
        col("majorId"),
        col("majorName"),
        current_timestamp().alias("created_at"),
        current_timestamp().alias("updated_at")
    )
)

df_major_silver.writeTo("nessie.silver_tables.major").using("iceberg").createOrReplace()
print(f"✅ MAJOR: {df_major_silver.count()} records")

## 4. Load Bảng SUBJECT_GROUP và SUBJECT

In [ ]:
# LOAD SUBJECT_GROUP & SUBJECT: Tổ hợp môn thi và danh sách môn học
df_tohop = spark.read.option("header", "true").option("encoding", "UTF-8").csv("s3a://bronze/structured_data/tohop_mon_fixed.csv")

# Subject Group
df_subject_group_silver = (
    df_tohop.select(
        col(df_tohop.columns[0]).cast("int").alias("subjectGroupId"),
        col(df_tohop.columns[1]).alias("subjectGroupName"),
        col(df_tohop.columns[2]).alias("subjectCombination"),
        current_timestamp().alias("created_at"),
        current_timestamp().alias("updated_at")
    )
    .filter(col("subjectGroupId").isNotNull() & col("subjectGroupName").isNotNull())
    .dropDuplicates(["subjectGroupName", "subjectCombination"])
)
df_subject_group_silver.writeTo("nessie.silver_tables.subject_group").using("iceberg").createOrReplace()

# Subject: Tách từ subjectCombination, dedupe theo lowercase
df_subject_silver = (
    df_tohop.select(explode(split(col(df_tohop.columns[2]), "-")).alias("subjectName"))
    .withColumn("subjectName", trim(col("subjectName")))
    .filter(col("subjectName").isNotNull() & (col("subjectName") != ""))
    .withColumn("subjectName_lower", lower(col("subjectName")))
    .dropDuplicates(["subjectName_lower"])
    .withColumn("subjectId", row_number().over(Window.orderBy("subjectName_lower")))
    .select(
        col("subjectId").cast("int"),
        col("subjectName"),
        current_timestamp().alias("created_at"),
        current_timestamp().alias("updated_at")
    )
)
df_subject_silver.writeTo("nessie.silver_tables.subject").using("iceberg").createOrReplace()
print(f"✅ SUBJECT_GROUP: {df_subject_group_silver.count()} | SUBJECT: {df_subject_silver.count()} records")

## 5. Load Bảng SELECTION_METHOD

In [ ]:
# LOAD SELECTION_METHOD: Phương thức xét tuyển (trích từ benchmark, loại bỏ "năm YYYY")
df_benchmark = spark.read.option("header", "true").option("encoding", "UTF-8").csv(
    "s3a://bronze/structured_data/điểm chuẩn các trường (2021-2025)/Điểm_chuẩn_các_ngành_đại_học_năm(2021-2025)*.csv"
)

df_selection_method_silver = (
    df_benchmark.select(trim(regexp_replace(col("PhuongThuc"), r"\s*năm\s+\d{4}.*$", "")).alias("selectionMethodName"))
    .filter(col("selectionMethodName").isNotNull() & (col("selectionMethodName") != ""))
    .distinct()
    .withColumn("selectionMethodId", row_number().over(Window.orderBy("selectionMethodName")))
    .select(
        col("selectionMethodId").cast("int"),
        col("selectionMethodName"),
        current_timestamp().alias("created_at"),
        current_timestamp().alias("updated_at")
    )
)
df_selection_method_silver.writeTo("nessie.silver_tables.selection_method").using("iceberg").createOrReplace()
print(f"✅ SELECTION_METHOD: {df_selection_method_silver.count()} records")

## 6. Load Bảng GradingScale

In [ ]:
# LOAD GRADING_SCALE: Phân loại thang điểm (trích từ benchmark, extract số từ description)
df_grading_scale_silver = (
    spark.read.option("header", "true").option("encoding", "UTF-8")
    .csv("s3a://bronze/structured_data/điểm chuẩn các trường (2021-2025)/Điểm_chuẩn_các_ngành_đại_học_năm(2021-2025)*.csv")
    .select(trim(col("PhanLoaiThangDiem")).alias("description"))
    .filter(col("description").isNotNull() & (col("description") != ""))
    .dropDuplicates(["description"])
    .withColumn("value", regexp_extract(col("description"), r"(\d+(?:\.\d+)?)", 1).cast("float"))
    .withColumn("gradingScaleId", monotonically_increasing_id().cast("int"))
    .select(
        "gradingScaleId",
        "value",
        "description",
        current_timestamp().alias("created_at"),
        current_timestamp().alias("updated_at")
    )
)
df_grading_scale_silver.writeTo("nessie.silver_tables.grading_scale").using("iceberg").createOrReplace()
print(f"✅ GRADING_SCALE: {df_grading_scale_silver.count()} records")

## 6. Load Bảng BENCHMARK

In [3]:
# LOAD BENCHMARK: Điểm chuẩn xét tuyển, join với lookup tables, group by trính trùng lặp
df_benchmark_raw = (
    spark.read.option("header", "true").option("encoding", "UTF-8")
    .csv("s3a://bronze/structured_data/điểm chuẩn các trường (2021-2025)/Điểm_chuẩn_các_ngành_đại_học_năm(2021-2025)*.csv")
    .withColumn("PhuongThuc_cleaned", trim(regexp_replace(col("PhuongThuc"), r"\s*năm\s+\d{4}.*$", "")))
)

# Join với lookup tables
df_benchmark_joined = (
    df_benchmark_raw
    .join(spark.table("nessie.silver_tables.selection_method"), 
          df_benchmark_raw["PhuongThuc_cleaned"] == col("selectionMethodName"), "left")
    .join(spark.table("nessie.silver_tables.subject_group"), 
          df_benchmark_raw["KhoiThi"] == col("subjectGroupName"), "left")
    .join(spark.table("nessie.silver_tables.grading_scale"), 
          trim(df_benchmark_raw["PhanLoaiThangDiem"]) == col("description"), "left")
    .select(
        col("MaTruong").cast("string").alias("schoolId"),
        col("MaNganh").cast("string").alias("majorId"),
        col("subjectGroupId").cast("int"),
        col("selectionMethodId").cast("int"),
        col("gradingScaleId").cast("int"),
        col("Nam").cast("int").alias("year"),
        col("DiemChuan").cast("double").alias("score")
    )
    .filter(
        col("schoolId").isNotNull() & col("majorId").isNotNull() &
        col("gradingScaleId").isNotNull() & col("year").isNotNull() &
        col("score").isNotNull() & col("selectionMethodId").isNotNull()
    )
)

# Group by để tránh trùng, lấy AVG score
df_benchmark_silver = (
    df_benchmark_joined
    .groupBy("schoolId", "majorId", "subjectGroupId", "selectionMethodId", "gradingScaleId", "year")
    .agg(round(avg("score"), 2).alias("score"))
    .withColumn("benchmarkId", row_number().over(
        Window.orderBy("schoolId", "majorId", "subjectGroupId", "selectionMethodId", "gradingScaleId", "year")
    ).cast("int"))
    .select(
        "benchmarkId", "schoolId", "majorId", "subjectGroupId", 
        "selectionMethodId", "gradingScaleId", "year", "score",
        current_timestamp().alias("created_at"),
        current_timestamp().alias("updated_at")
    )
)

df_benchmark_silver.writeTo("nessie.silver_tables.benchmark").using("iceberg").createOrReplace()
print(f"✅ BENCHMARK: {df_benchmark_silver.count()} records")

[Stage 13:>                                                         (0 + 2) / 2]

✅ BENCHMARK: 163399 records


## 7. Load Bảng REGION

In [ ]:
# LOAD REGION: Danh sách vùng thi, format regionId thành 2 chữ số (01, 02, ...)
df_region = spark.read.option("header", "true").option("encoding", "UTF-8").csv("s3a://bronze/structured_data/region.csv")
df_region_silver = (
    df_region.select(
        lpad(col(df_region.columns[0]).cast("string"), 2, "0").alias("regionId"),
        col(df_region.columns[1]).alias("regionName"),
        current_timestamp().alias("created_at"),
        current_timestamp().alias("updated_at")
    )
    .filter(col("regionId").isNotNull() & col("regionName").isNotNull())
    .dropDuplicates(["regionId"])
)
df_region_silver.writeTo("nessie.silver_tables.region").using("iceberg").createOrReplace()
print(f"✅ REGION: {df_region_silver.count()} records")

## 8. Load Bảng STUDENT_SCORES

In [ ]:
# LOAD STUDENT_SCORES: Điểm thi từng thí sinh (2021-2025), parse scores thành Map<subjectId, score>
df_scores = None
for year in range(2021, 2026):
    try:
        df_year = (
            spark.read.option("header", "true").option("encoding", "UTF-8")
            .csv(f"s3a://bronze/structured_data/điểm từng thí sinh/{year}/*.csv")
            .withColumn("Year", lit(year))
        )
        df_scores = df_year if df_scores is None else df_scores.union(df_year)
    except:
        pass

# Tạo subject map để convert tên môn -> subjectId
subject_map = {row.subjectName: row.subjectId for row in 
               spark.table("nessie.silver_tables.subject").select("subjectId", "subjectName").collect()}

# UDF parse scores: "Toán:8.5,Văn:7.0" -> {1: 8.5, 2: 7.0}
def parse_scores(score_str: str) -> Dict[int, float]:
    if not score_str or not score_str.strip(): return {}
    result = {}
    try:
        for pair in score_str.split(","):
            if ":" in pair:
                name, score = pair.split(":")
                if name.strip() in subject_map:
                    try:
                        result[subject_map[name.strip()]] = float(score.strip())
                    except:
                        pass
    except:
        pass
    return result

parse_scores_udf = udf(parse_scores, MapType(IntegerType(), DoubleType()))

df_student_scores_silver = (
    df_scores
    .withColumn("studentId", concat(col("SBD"), col("Year").cast("string")))
    .withColumn("regionId", substring(col("SBD"), 1, 2))
    .withColumn("scores", parse_scores_udf(col("DiemThi")))
    .select(
        col("studentId"),
        col("regionId"),
        col("Year").cast("int").alias("year"),
        col("scores"),
        current_timestamp().alias("created_at"),
        current_timestamp().alias("updated_at")
    )
    .filter(col("studentId").isNotNull() & col("year").isNotNull() & col("scores").isNotNull())
    .dropDuplicates(["studentId"])
)

df_student_scores_silver.writeTo("nessie.silver_tables.student_scores").using("iceberg").createOrReplace()
print(f" STUDENT_SCORES: {df_student_scores_silver.count():,} records")

## 9. Load Bảng ARTICLE và COMMENT từ TikTok Data

In [ ]:
# LOAD TIKTOK ARTICLE & COMMENT: Parse CSV files với metadata ở đầu, comment ở cuối
def clean_url(url: str):
    """Loại bỏ prefix URL TikTok"""
    return url.replace("https://www.tiktok.com/", "") if url else url

def clean_reply_to(name: str):
    """Loại bỏ ?lang=vi-VN suffix"""
    return name.replace("?lang=vi-VN", "") if name and name.endswith("?lang=vi-VN") else name

def parse_number(num_str: str) -> int:
    """Parse số có thể chứa 'K' (nghìn)"""
    try:
        if not num_str or num_str == "N/A": return 0
        return int(float(num_str.replace("K", "")) * 1000) if "K" in num_str else int(num_str)
    except:
        return 0

# Lấy danh sách files TikTok
tiktok_path = "s3a://bronze/MangXaHoi/tiktok-data/comments/*.csv"
tiktok_files = [row.source_file for row in 
                spark.read.option("header", "false").option("encoding", "UTF-8").csv(tiktok_path)
                .withColumn("source_file", input_file_name()).select("source_file").distinct().collect()]

all_articles, all_comments = [], []
article_id, comment_id = 1, 1

for file_path in tiktok_files:
    rows = spark.read.option("header", "false").option("encoding", "UTF-8").csv(file_path).collect()
    
    # Parse metadata từ 20 dòng đầu
    metadata = {"post_url": "", "author": "", "timePublish": datetime.now(), 
                "likeCount": 0, "commentCount": 0, "shareCount": 0, "title": ""}
    
    for i, row in enumerate(rows[:20]):
        line = row[0] if row[0] else ""
        if "Post URL:" in line: metadata["post_url"] = line.split("Post URL:")[1].strip()
        elif "Người đăng:" in line: metadata["author"] = line.split("Người đăng:")[1].strip()
        elif "Thời gian đăng:" in line:
            try: metadata["timePublish"] = datetime.strptime(line.split("Thời gian đăng:")[1].strip(), "%d-%m-%Y")
            except: pass
        elif "Số lượt tym:" in line: metadata["likeCount"] = parse_number(line.split("Số lượt tym:")[1].strip())
        elif "Số lượt comment:" in line: metadata["commentCount"] = parse_number(line.split("Số lượt comment:")[1].strip())
        elif "Số lượt share:" in line: metadata["shareCount"] = parse_number(line.split("Số lượt share:")[1].strip())
        elif "Mô tả của bài đăng:" in line:
            desc = line.split("Mô tả của bài đăng:")[1].strip()
            if desc.endswith('"') and desc.count('"') >= 2:
                metadata["title"] = desc.strip('"')
            else:
                desc_lines = [desc.lstrip('"')]
                for j in range(i+1, len(rows)):
                    next_line = rows[j][0] if rows[j][0] else ""
                    desc_lines.append(next_line)
                    if next_line.endswith('"'): break
                metadata["title"] = "\n".join(desc_lines).strip().strip('"')
    
    all_articles.append({
        "articleID": article_id, "title": metadata["title"], "description": None,
        "author": metadata["author"], "url": clean_url(metadata["post_url"]),
        "timePublish": metadata["timePublish"], "likeCount": metadata["likeCount"],
        "commentCount": metadata["commentCount"], "shareCount": metadata["shareCount"],
        "type": "TikTok", "created_at": datetime.now(), "updated_at": datetime.now()
    })
    
    # Parse comments bằng CSV reader
    text_lines = [r.value or "" for r in spark.read.text(file_path).collect()]
    header_idx = next((i for i, line in enumerate(text_lines) if line.startswith("STT,") and "Comment" in line), -1)
    
    if header_idx >= 0:
        csv_str = "\n".join(text_lines[header_idx:])
        for row in csv.DictReader(io.StringIO(csv_str)):
            try:
                all_comments.append({
                    "commentID": comment_id, "articleID": article_id,
                    "name": row.get("Tên", ""), "tagName": row.get("Tag tên", ""),
                    "urlUser": clean_url(row.get("URL", "")), "comment": row.get("Comment", ""),
                    "commentTime": datetime.strptime(row.get("Time", ""), "%d-%m-%Y") if row.get("Time") else datetime.now(),
                    "commentLike": int(row.get("Likes", "0")) if row.get("Likes", "").isdigit() else 0,
                    "levelComment": 2 if row.get("Level Comment") == "Yes" else 1,
                    "replyTo": clean_reply_to(row.get("Replied To Tag Name", "")) if row.get("Replied To Tag Name") not in ["", "---"] else None,
                    "numberOfReply": int(row.get("Number of Replies", "0")) if row.get("Number of Replies", "").isdigit() else 0,
                    "created_at": datetime.now(), "updated_at": datetime.now()
                })
                comment_id += 1
            except: pass
    article_id += 1

# Ghi vào Silver
article_schema = StructType([StructField("articleID", IntegerType(), False), StructField("title", StringType(), True),
    StructField("description", StringType(), True), StructField("author", StringType(), True), StructField("url", StringType(), True),
    StructField("timePublish", TimestampType(), True), StructField("likeCount", IntegerType(), True),
    StructField("commentCount", IntegerType(), True), StructField("shareCount", IntegerType(), True),
    StructField("type", StringType(), True), StructField("created_at", TimestampType(), True), StructField("updated_at", TimestampType(), True)])

spark.createDataFrame(all_articles, article_schema).writeTo("nessie.silver_tables.article").using("iceberg").createOrReplace()

comment_schema = StructType([StructField("commentID", IntegerType(), False), StructField("articleID", IntegerType(), True),
    StructField("name", StringType(), True), StructField("tagName", StringType(), True), StructField("urlUser", StringType(), True),
    StructField("comment", StringType(), True), StructField("commentTime", TimestampType(), True), StructField("commentLike", IntegerType(), True),
    StructField("levelComment", IntegerType(), True), StructField("replyTo", StringType(), True), StructField("numberOfReply", IntegerType(), True),
    StructField("created_at", TimestampType(), True), StructField("updated_at", TimestampType(), True)])

spark.createDataFrame(all_comments, comment_schema).writeTo("nessie.silver_tables.comment").using("iceberg").createOrReplace()
print(f" TIKTOK - ARTICLE: {len(all_articles)} | COMMENT: {len(all_comments)} records")

In [ ]:
# Tạo DataFrame cho article
if all_articles:
    article_schema = StructType([
        StructField("articleID", IntegerType(), False),
        StructField("title", StringType(), True),
        StructField("description", StringType(), True),
        StructField("author", StringType(), True),
        StructField("url", StringType(), True),
        StructField("timePublish", TimestampType(), True),
        StructField("likeCount", IntegerType(), True),
        StructField("commentCount", IntegerType(), True),
        StructField("shareCount", IntegerType(), True),
        StructField("type", StringType(), True),
        StructField("created_at", TimestampType(), True),
        StructField("updated_at", TimestampType(), True)
    ])
    
    df_article_silver = spark.createDataFrame(all_articles, article_schema)
    
    # Ghi vào bảng article
    df_article_silver.writeTo("nessie.silver_tables.article").using("iceberg").createOrReplace()
    print(f"Đã ghi {df_article_silver.count()} dòng vào bảng article")
    
    # Verify
    # spark.table("nessie.silver_tables.article").show(5, truncate=False)
else:
    print("Không có dữ liệu article để ghi")

In [ ]:
# Tạo DataFrame cho comment
if all_comments:
    comment_schema = StructType([
        StructField("commentID", IntegerType(), False),
        StructField("articleID", IntegerType(), True),
        StructField("name", StringType(), True),
        StructField("tagName", StringType(), True),
        StructField("urlUser", StringType(), True),
        StructField("comment", StringType(), True),
        StructField("commentTime", TimestampType(), True),
        StructField("commentLike", IntegerType(), True),
        StructField("levelComment", IntegerType(), True),
        StructField("replyTo", StringType(), True),
        StructField("numberOfReply", IntegerType(), True),
        StructField("created_at", TimestampType(), True),
        StructField("updated_at", TimestampType(), True)
    ])
    
    df_comment_silver = spark.createDataFrame(all_comments, comment_schema)
    
    # Ghi vào bảng comment
    df_comment_silver.writeTo("nessie.silver_tables.comment").using("iceberg").createOrReplace()
    print(f"Đã ghi {df_comment_silver.count()} dòng vào bảng comment")
    
    # Verify
    spark.table("nessie.silver_tables.comment").show(5, truncate=False)
    
    # Thống kê theo article
    print("\nThống kê comment theo article:")
    spark.table("nessie.silver_tables.comment").groupBy("articleID").count().orderBy("articleID").show()
else:
    print("Không có dữ liệu comment để ghi")

In [ ]:
from pyspark.sql.functions import col, trim, current_timestamp

print("=" * 80)
print("CẬP NHẬT DESCRIPTION CHO BẢNG ARTICLE")
print("=" * 80)

# ===== 1. Đọc file CSV mô tả =====
CSV_PATH = "s3a://bronze/MangXaHoi/Tiktok-data/sub/*.csv"  # sửa path cho đúng

df_desc_raw = (
    spark.read
        .option("header", "true")
        .option("inferSchema", "false")
        .option("encoding", "UTF-8")
        .csv(CSV_PATH)
)

print(f"Đọc được {df_desc_raw.count():,} dòng từ file mô tả")

# Chuẩn hoá cột ID & Description
df_desc = df_desc_raw.select(
    trim(col("url")).alias("url"),              # ID trùng với cột url của bảng article
    trim(col("sub")).alias("new_description")
).filter(col("url").isNotNull() & (col("url") != ""))

print(f"Số dòng dùng để cập nhật: {df_desc.count():,}")

# Tạo temp view để dùng trong SQL
df_desc.createOrReplaceTempView("article_desc_update")

# ===== 2. MERGE vào bảng article =====
# Chỉ cập nhật cột description (và updated_at cho tiện)
spark.sql("""
MERGE INTO nessie.silver_tables.article AS a
USING article_desc_update AS d
ON a.url = d.url
WHEN MATCHED THEN UPDATE SET
  a.description = d.new_description,
  a.updated_at = current_timestamp()
""")

print("Đã cập nhật description cho bảng article!")


In [ ]:
#### Load Facebook data

In [ ]:
import re
from datetime import datetime
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# Map tháng tiếng Việt -> số
MONTH_MAP = {
    "Tháng 1": "01", "Tháng 2": "02", "Tháng 3": "03", "Tháng 4": "04",
    "Tháng 5": "05", "Tháng 6": "06", "Tháng 7": "07", "Tháng 8": "08",
    "Tháng 9": "09", "Tháng 10": "10", "Tháng 11": "11", "Tháng 12": "12"
}

def parse_vietnam_datetime(dt_str):
    """
    Convert:
    'Thứ Sáu, 1 Tháng 8, 2025 lúc 20:18'
          ↓
    '2025-08-01 20:18:00'
    """
    if not dt_str:
        return None
    
    try:
        # Bỏ tiền tố thứ ngày
        # Ví dụ: "Thứ Sáu, " → ""
        dt_str = dt_str.split(",", 1)[1].strip()

        # dt_str còn lại:
        # "1 Tháng 8, 2025 lúc 20:18"

        # Tách ngày – tháng tiếng Việt
        # 1 Tháng 8
        match = re.search(r"(\d+)\s+(Tháng\s+\d+)", dt_str)
        if not match:
            return None

        day = match.group(1)
        month_text = match.group(2)
        month = MONTH_MAP.get(month_text)

        # Lấy năm
        year_match = re.search(r",\s*(\d{4})", dt_str)
        if not year_match:
            return None
        year = year_match.group(1)

        # Lấy giờ phút
        time_match = re.search(r"lúc\s+(\d{1,2}:\d{2})", dt_str)
        if time_match:
            time_str = time_match.group(1)
        else:
            time_str = "00:00"

        # Format chuẩn: yyyy-MM-dd HH:mm:ss
        final_str = f"{year}-{month}-{int(day):02d} {time_str}:00"
        return final_str

    except Exception:
        return None


parse_vn_time_udf = udf(parse_vietnam_datetime, StringType())


In [ ]:
# spark.sql("""
# DELETE FROM nessie.silver_tables.article
# WHERE type = 'facebook'
# """)


In [ ]:
# spark.sql("""
# DELETE FROM nessie.silver_tables.comment
# WHERE articleID NOT IN (
#     SELECT articleID FROM nessie.silver_tables.article
# )
# """)


In [ ]:
from pyspark.sql import Window
from pyspark.sql.functions import (
    col, trim, current_timestamp, to_timestamp,
    row_number, lit, max as F_max
)

print("=" * 80)
print("LOAD DỮ LIỆU FACEBOOK VÀO BẢNG ARTICLE & COMMENT (KHÔNG TRÙNG ID)")
print("=" * 80)

posts_path = "s3a://bronze/MangXaHoi/Face-data/posts_FB.csv"
comments_path = "s3a://bronze/MangXaHoi/Face-data/comments_FB.csv"

# ====================================================
# 0. LẤY MAX ID HIỆN CÓ TRONG BẢNG
# ====================================================
# Nếu bảng rỗng thì maxID = None -> dùng 0
try:
    max_article_row = (
        spark.table("nessie.silver_tables.article")
             .agg(F_max("articleID").alias("maxID"))
             .collect()[0]
    )
    base_article_id = max_article_row["maxID"] or 0
except Exception:
    base_article_id = 0

try:
    max_comment_row = (
        spark.table("nessie.silver_tables.comment")
             .agg(F_max("commentID").alias("maxID"))
             .collect()[0]
    )
    base_comment_id = max_comment_row["maxID"] or 0
except Exception:
    base_comment_id = 0

print(f"base_article_id (offset) = {base_article_id}")
print(f"base_comment_id (offset) = {base_comment_id}")

# ====================================================
# 1. ĐỌC BẢNG POSTS_FB -> ARTICLE
# ====================================================
df_posts_raw = (
    spark.read
        .option("header", "true")
        .option("inferSchema", "false")
        .option("encoding", "UTF-8")
        .csv(posts_path)
)

print(f"Đọc được {df_posts_raw.count():,} dòng từ posts_FB")

df_posts_clean = df_posts_raw.select(
    trim(col("ID")).alias("url_post"),      # URL bài viết (dùng để join)
    trim(col("Title")).alias("title"),
    trim(col("Description")).alias("description"),
    trim(col("Author")).alias("author"),
    trim(col("Url")).alias("url_extra"),
    trim(col("TimePublish")).alias("TimePublish_raw"),
    trim(col("Like")).alias("Like_raw"),
    trim(col("Share")).alias("Share_raw"),
    trim(col("Comment")).alias("Comment_raw")
)

df_posts_parsed = (
    df_posts_clean
        # Chuẩn hoá string tiếng Việt -> 'yyyy-MM-dd HH:mm:ss'
        .withColumn("TimePublish_clean", parse_vn_time_udf(col("TimePublish_raw")))
        # Convert string chuẩn -> timestamp
        .withColumn("timePublish", to_timestamp(col("TimePublish_clean")))
        .withColumn("likeCount", col("Like_raw").cast("int"))
        .withColumn("shareCount", col("Share_raw").cast("int"))
        .withColumn("commentCount", col("Comment_raw").cast("int"))
)


# Window để đánh số local trong batch FB
w_article = Window.orderBy("url_post")

df_article_silver = (
    df_posts_parsed
        # articleID_local: 1,2,3,... trong batch Facebook
        .withColumn("articleID_local", row_number().over(w_article))
        # articleID thật = offset + local
        .withColumn(
            "articleID",
            (col("articleID_local") + lit(base_article_id)).cast("int")
        )
        .select(
            col("articleID"),
            col("title"),
            col("description"),
            col("author"),
            col("url_post").alias("url"),
            col("timePublish"),
            col("likeCount"),
            col("commentCount"),
            col("shareCount"),
            lit("facebook").alias("type"),
            current_timestamp().alias("created_at"),
            current_timestamp().alias("updated_at")
        )
)

print("Mẫu dữ liệu article_silver:")
df_article_silver.show(5, truncate=False)

# Ghi vào bảng article
df_article_silver.writeTo("nessie.silver_tables.article").using("iceberg").append()
print(f"Đã ghi {df_article_silver.count():,} dòng vào nessie.silver_tables.article")

# Tạo lookup cho batch FB (chỉ cần url & articleID của batch này)
df_article_fb_lookup = df_article_silver.select("articleID", "url").distinct()

# ====================================================
# 2. ĐỌC BẢNG COMMENT_FB -> COMMENT, JOIN THEO URL
# ====================================================
df_cmt_raw = (
    spark.read
        .option("header", "true")
        .option("inferSchema", "false")
        .option("encoding", "UTF-8")
        .csv(comments_path)
)

print(f"Đọc được {df_cmt_raw.count():,} dòng từ comment_FB")

# comment_FB: STT,Id_post,Comment
df_cmt_clean = df_cmt_raw.select(
    trim(col("STT")).alias("stt"),
    trim(col("Id_post")).alias("post_url"),
    trim(col("Comment")).alias("comment_text")
).filter(col("post_url").isNotNull() & (col("post_url") != ""))

# JOIN comment với batch article FB vừa tạo (không join toàn bộ bảng để tránh nhầm nguồn khác)
df_cmt_joined = (
    df_cmt_clean.alias("c")
        .join(
            df_article_fb_lookup.alias("a"),
            col("c.post_url") == col("a.url"),
            "inner"
        )
)

print(f"Số dòng comment match với article FB: {df_cmt_joined.count():,}")

# Window để đánh số local comment trong batch FB
w_comment = Window.orderBy(col("a.articleID"), col("c.stt"))

df_comment_silver = (
    df_cmt_joined
        .withColumn("commentID_local", row_number().over(w_comment))
        .withColumn(
            "commentID",
            (col("commentID_local") + lit(base_comment_id)).cast("int")
        )
        .select(
            col("commentID"),
            col("a.articleID").alias("articleID"),
            lit(None).cast("string").alias("name"),
            lit(None).cast("string").alias("tagName"),
            lit(None).cast("string").alias("urlUser"),
            col("comment_text").alias("comment"),
            lit(None).cast("timestamp").alias("commentTime"),
            lit(None).cast("int").alias("commentLike"),
            lit(1).cast("int").alias("levelComment"),
            lit(None).cast("string").alias("replyTo"),
            lit(0).cast("int").alias("numberOfReply"),
            current_timestamp().alias("created_at"),
            current_timestamp().alias("updated_at")
        )
)

print("Mẫu dữ liệu comment_silver:")
df_comment_silver.show(5, truncate=False)

df_comment_silver.writeTo("nessie.silver_tables.comment").using("iceberg").append()
print(f"Đã ghi {df_comment_silver.count():,} dòng vào nessie.silver_tables.comment")

# ====================================================
# 3. CHECK LẠI
# ====================================================
print("\nCHECK lại article (facebook):")
spark.table("nessie.silver_tables.article").where("type = 'facebook'").show(5, truncate=False)

print("\nCHECK lại comment (facebook, join với article):")
spark.sql("""
SELECT c.commentID, c.articleID, a.title, c.comment
FROM nessie.silver_tables.comment c
JOIN nessie.silver_tables.article a
  ON c.articleID = a.articleID
WHERE a.type = 'facebook'
LIMIT 10
""").show(truncate=False)


## 12. Dừng Spark Session

In [ ]:
# Dừng Spark Session để giải phóng resources
spark.stop()
print(" Spark Session đã được dừng!")